In [1]:
# Import modules

%matplotlib notebook
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from graph_model import CallBacks, DataGenerator, GraphNet

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import pickle
import shutil

In [2]:
tf.config.set_visible_devices([], 'GPU')

In [3]:
# Define the data directories

model_name = "max-aggregate2_dropout=0.15+Gaussian"

cwd = os.getcwd()
data_dir = os.path.join(cwd, "data")
processed_waveform_dir = os.path.join(data_dir, "waveforms_proc_broadband")

# Data preparation

In [4]:
# Fix random seed for reproducibility
np.random.seed(0)

# Data ranges (min/max)
minlatitude = 32
maxlatitude = 36
minlongitude = -120
maxlongitude = -116
maxdepth = 30e3
minmag = 3
maxmag = 6

# Fraction of the data to be used in training
split = 0.8

As part of the data preparation process, we scale the data within the range of $\pm 1$, and split the data set into a training set and a validation set.

In [5]:
# Get the event catalogue
catalogue = pd.read_csv(os.path.join(data_dir, "catalogue.csv"))[["lat", "lon", "depth", "mag"]]

# Check which events have data
for i, event in catalogue.iterrows():
    # Event data file
    cat_file = os.path.join(processed_waveform_dir, "%d.npy" % i)
    # If file not exists: remove event from catalogue
    if not os.path.isfile(cat_file):
        catalogue.drop(index=i, inplace=True)

# Print number of events
print(len(catalogue))        

# Event identifiers
ids = catalogue.index.values
# Uniform weights
weights = np.ones((len(ids), 1))
# Event lat/lon/depth/magnitude
catalogue = catalogue.values

# Scale data
catalogue[:, 0] = (catalogue[:, 0] - minlatitude) / (maxlatitude - minlatitude)
catalogue[:, 1] = (catalogue[:, 1] - minlongitude) / (maxlongitude - minlongitude)
catalogue[:, 2] = catalogue[:, 2] / maxdepth
catalogue[:, 3] = (catalogue[:, 3] - minmag) / (maxmag - minmag)

catalogue = (catalogue - 0.5) * 2

# Concatenate identifiers and weights to event data
catalogue = np.concatenate([ids.reshape(-1, 1), weights.reshape(-1, 1), catalogue], axis=1)

# Randomly split events into train and validation sets
inds = np.arange(catalogue.shape[0])
np.random.shuffle(inds)
N_split = int(split * catalogue.shape[0])

train_inds = inds[:N_split]
test_inds = inds[N_split:]

# Split catalogue
train_catalogue = catalogue[train_inds]
test_catalogue = catalogue[test_inds]

# Check data distributions to ensure that train and validation sets are similar!
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(9, 7))
axes = axes.ravel()

for i in range(4):
    ax = axes[i]
    ax.hist(catalogue[:, i+2], bins=20, density=False)
    ax.hist(train_catalogue[:, i+2], bins=20, alpha=0.5, density=False)
    ax.hist(test_catalogue[:, i+2], bins=20, alpha=0.5, density=False)

plt.tight_layout()    
plt.show()

1377


<IPython.core.display.Javascript object>

In [6]:
# Read seismic station information
stations = pd.read_csv(os.path.join(data_dir, "stations.csv"))[["code", "lat", "lon"]]
# Scale station geographic locations
stations["lat"] = ((stations["lat"] - minlatitude) / (maxlatitude - minlatitude) - 0.5) * 2
stations["lon"] = ((stations["lon"] - minlongitude) / (maxlongitude - minlongitude) - 0.5) * 2

In [7]:
# Open the event-station lookup file
lookup_file = os.path.join(data_dir, "catalogue_station_lookup_final.pickle")

with open(lookup_file, "rb") as f:
    lookup = pickle.load(f)

In [8]:
# Number of stations per sample
N_sub = 50
# Number of time sample points
N_t = 2048

# Instantiate train data generator
train_generator = DataGenerator(
    data_dir=processed_waveform_dir,
    catalogue=train_catalogue,
    stations=stations,
    lookup=lookup,
    N_sub=N_sub,
    N_t=N_t,
    batch_size=32,
)

# Instantiate validation data generator
test_generator = DataGenerator(
    data_dir=processed_waveform_dir,
    catalogue=test_catalogue,
    stations=stations,
    lookup=lookup,
    N_sub=N_sub,
    N_t=N_t,
    batch_size=32,
)

# Model construction

In [9]:
""" Define Tensorflow/Keras callbacks """

# Tensorboard log directory
logdir = os.path.join("logs", model_name)
if os.path.isdir(logdir):
    shutil.rmtree(logdir)

# Save model directory
savedir = os.path.join("save", model_name)
if not os.path.isdir(savedir):
    os.makedirs(savedir)

# Best and last model save files
savefile_best = "best-model.h5"
savefile_last = "last-model.h5"

# Instantiate callbacks
tensorboard_callback = CallBacks.tensorboard(logdir)
checkpoint_callback_best = CallBacks.checkpoint(os.path.join(savedir, savefile_best), best=True)
checkpoint_callback_last = CallBacks.checkpoint(os.path.join(savedir, savefile_last), best=False)

In [10]:
# Instantiate Graph Neural Network
graph = GraphNet()
# Overwrite default model parameters
params = {
    "N_t": N_t,
    "dropout_rate": 0.15,
    "activation": "relu",
}
# Set parameters
graph.set_params(params)
# Construct a graph model
model = graph.construct()
# Print out a summary
model.summary()
# Output model summary to a PNG image (if possible)
#plot_model(model, show_shapes=True)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50, 2048, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 50, 2048, 4)  64          input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 50, 2048, 4)  0           conv2d[0][0]                     
__________________________________________________________________________________________________
spatial_dropout2d (SpatialDropo (None, 50, 2048, 4)  0           activation[0][0]                 
______________________________________________________________________________________________

In [ ]:
# Compile the model with an ADAM optimiser and mean absolute error loss function
model.compile(
    optimizer=Adam(learning_rate=graph.LR),
    loss="mean_absolute_error",
)

# Train the model for 500 epochs
model.fit(
    x=train_generator,
    validation_data=test_generator,
    callbacks=[tensorboard_callback, checkpoint_callback_best, checkpoint_callback_last],
    verbose=2, epochs=500,
)

Epoch 1/500
34/34 - 20s - loss: 0.4533 - val_loss: 0.3855
Epoch 2/500
34/34 - 19s - loss: 0.3704 - val_loss: 0.3472
Epoch 3/500
34/34 - 18s - loss: 0.3484 - val_loss: 0.3402
Epoch 4/500
34/34 - 19s - loss: 0.3348 - val_loss: 0.3205
Epoch 5/500
34/34 - 19s - loss: 0.3260 - val_loss: 0.3192
Epoch 6/500
34/34 - 19s - loss: 0.3119 - val_loss: 0.2995
Epoch 7/500
34/34 - 19s - loss: 0.2957 - val_loss: 0.2818
Epoch 8/500
34/34 - 19s - loss: 0.2749 - val_loss: 0.2547
Epoch 9/500
34/34 - 19s - loss: 0.2596 - val_loss: 0.2441
Epoch 10/500
34/34 - 19s - loss: 0.2485 - val_loss: 0.2370
Epoch 11/500
34/34 - 19s - loss: 0.2428 - val_loss: 0.2216
Epoch 12/500
34/34 - 19s - loss: 0.2315 - val_loss: 0.2201
Epoch 13/500
34/34 - 19s - loss: 0.2270 - val_loss: 0.2120
Epoch 14/500
34/34 - 19s - loss: 0.2266 - val_loss: 0.2111
Epoch 15/500
34/34 - 19s - loss: 0.2195 - val_loss: 0.2039
Epoch 16/500
34/34 - 19s - loss: 0.2156 - val_loss: 0.2006
Epoch 17/500
34/34 - 19s - loss: 0.2115 - val_loss: 0.2013
Epoch 